<a href="https://colab.research.google.com/github/1on1pt/JPX_Tokyo_Stock_Exchange_Prediction/blob/Jerri/pyspark_to_postgres.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.1.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease [1,581 B]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Packages [748 kB]
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:6 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:9 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ Packages [85.2 kB]
Get:10 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:12 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Get:13 http://security.ubuntu.com/ubuntu bionic-sec

In [2]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2022-05-14 14:20:54--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  1.23MB/s    in 0.8s    

2022-05-14 14:20:55 (1.23 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M16-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [7]:
prices_df = spark.read.options(header='True', inferSchema='True', delimiter=',') \
  .csv("prices_clean.csv")

In [8]:
prices_df.show()

+-------------+----------+--------------+------+------+------+------+-------+---------------+-------------------+
|        RowId|      Date|SecuritiesCode|  Open|  High|   Low| Close| Volume|SupervisionFlag|             Target|
+-------------+----------+--------------+------+------+------+------+-------+---------------+-------------------+
|20170104_1301|2017-01-04|          1301|2734.0|2755.0|2730.0|2742.0|  31400|          false|  7.304601899196E-4|
|20170104_1332|2017-01-04|          1332| 568.0| 576.0| 563.0| 571.0|2798500|          false| 0.0123239436619717|
|20170104_1333|2017-01-04|          1333|3150.0|3210.0|3140.0|3210.0| 270800|          false| 0.0061538461538461|
|20170104_1376|2017-01-04|          1376|1510.0|1550.0|1510.0|1550.0|  11300|          false| 0.0110533159947985|
|20170104_1377|2017-01-04|          1377|3270.0|3350.0|3270.0|3330.0| 150800|          false| 0.0030257186081694|
|20170104_1379|2017-01-04|          1379|2105.0|2147.0|2101.0|2143.0|  77300|          f

In [15]:
# Configure settings for RDS
mode = "append"
jdbc_url="jdbc:postgresql://tokyose.ctcjkv5b73cy.us-east-1.rds.amazonaws.com:5432/tokyo_se"
config = {"user":"bootcamp", 
          "password": "tokyoBull$", 
          "driver":"org.postgresql.Driver"}

In [16]:
# Write prices_df to table in RDS
prices_df.write.jdbc(url=jdbc_url, table='prices_table', mode=mode, properties=config)

**financials data**

In [17]:
#create df from cleaned csv
financials_df = spark.read.options(header='True', inferSchema='True', delimiter=',') \
  .csv("financials_clean.csv")

In [19]:
#show df
financials_df.show()

+----------------+----------+--------------+------------+----------------+------------+------------------------+-----------------------------+
|disclosurenumber|      date|securitiescode|    netsales|earningspershare|      equity|forecastearningspershare|materialchangesinsubsidiaries|
+----------------+----------+--------------+------------+----------------+------------+------------------------+-----------------------------+
|  20211203446434|2021-12-03|          2929| 12606000000|           42.45|  8788000000|                  138.36|                        false|
|  20211203446277|2021-12-03|          9627|152972000000|           94.68|115810000000|                  234.28|                        false|
|  20211203446219|2021-12-03|          3657| 24334000000|           42.87| 17065000000|                   56.53|                        false|
|  20211202446055|2021-12-03|          2353|  6286000000|            2.82| 10198000000|                    8.91|                        false|

In [20]:
#write data to RDS for financials
financials_df.write.jdbc(url=jdbc_url, table='financials_table', mode=mode, properties=config)